# Example code to apply the modular splicing model

Example test variants come from ClinVar BRCA1 variants

## Splicing delta PSI prediction

In [1]:
from mmsplice.vcf_dataloader import SplicingVCFDataloader
from mmsplice import MMSplice, predict_all_table

from mmsplice.utils import max_varEff

Using TensorFlow backend.


In [2]:
gtf = '../tests/data/test.gtf'
vcf = '../tests/data/test.vcf.gz'
fasta = '../tests/data/hg19.nochr.chr17.fa'
gtfIntervalTree = '../tests/data/test.pkl' # pickle exon interval Tree

In [3]:
dl = SplicingVCFDataloader(gtfIntervalTree, 
                          fasta,
                          vcf,
                          out_file=gtfIntervalTree,
                          split_seq=False, overhang=(100,100))

In [4]:
next(dl)

{'inputs': {'seq': 'CCTAAGAACTCATACAACCAGGACCCTGGAGTCGATTGATTAGAGCCTAGTCCAGGAGAATGAATTGACACTAATCTCTGCTTGTGTTCTCTGTCTCCAGCAATTGGGCAGATGTGTGAGGCACCTGTGGTGACCCGAGAGTGGGTGTTGGACAGTGTAGCACTCTACCAGTGCCAGGAGCTGGACACCTACCTGATACCCCAGATCCCCCACAGCCACTACTGACTGCAGCCAGCCACAGGTACAGAGCCACAGGACCCCAAGAATGAGCTTACAAAGTGGCCTTTCCAGGCCCTGGGAGCTCCTCTCACTCTTCAGTCCTTCT',
  'intronl_len': 100,
  'intronr_len': 100},
 'inputs_mut': {'seq': 'CCTAAGAACTCATACAACCAGGACCCTGGAGTCGATTGATTAGAGCCTAGTCCAGGAGAATGAATTGACACTAATCTCTGCTTGTGTTCTCTGTCTCCAGCAATTGTGTGAGGCACCTGTGGTGACCCGAGAGTGGGTGTTGGACAGTGTAGCACTCTACCAGTGCCAGGAGCTGGACACCTACCTGATACCCCAGATCCCCCACAGCCACTACTGACTGCAGCCAGCCACAGGTACAGAGCCACAGGACCCCAAGAATGAGCTTACAAAGTGGCCTTTCCAGGCCCTGGGAGCTCCTCTCACTCTTCAGTCCTTCT',
  'intronl_len': 100,
  'intronr_len': 100},
 'metadata': {'ranges': GenomicRanges(chr='17', start=41197595, end=41197919, id='ENST00000461221', strand='-'),
  'variant': {'CHROM': '17',
   'POS': 41197805,
   'ID': '182075',
   'REF': 'ACATCTGCC',
   'ALT': 'A',

In [5]:
model = MMSplice(
    exon_cut_l=0,
    exon_cut_r=0,
    acceptor_intron_cut=6,
    donor_intron_cut=6,
    acceptor_intron_len=50,
    acceptor_exon_len=3,
    donor_exon_len=5,
    donor_intron_len=13)

/opt/modules/i12g/anaconda/3-5.0.1/envs/splicing/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
predictions = predict_all_table(model, dl, batch_size=1024, split_seq=False, assembly=True)

0it [00:00, ?it/s]/opt/modules/i12g/anaconda/3-5.0.1/envs/splicing/lib/python3.6/site-packages/mmsplice-0.2.2-py3.6.egg/mmsplice/mmsplice.py:159: UserWarning: None GT donor
  warnings.warn("None GT donor", UserWarning)
/opt/modules/i12g/anaconda/3-5.0.1/envs/splicing/lib/python3.6/site-packages/mmsplice-0.2.2-py3.6.egg/mmsplice/mmsplice.py:161: UserWarning: None AG donor
  warnings.warn("None AG donor", UserWarning)
3it [00:35, 11.78s/it]


In [7]:
predictionsMax = max_varEff(predictions)

In [9]:
predictionsMax.sort_values(['mmsplice_dlogitPsi']).head()

,ID,mmsplice_dlogitPsi,exons
632,17:41242891:GTGGGATACATACTACTGAATGCAAAGGACACCA...,-13.616748,17_41242961_41243049:-
957,17:41251814:CTTTTGAGGTTGTATCCGCTGCTTTGTCCTCAGA...,-10.040847,17_41251848_41251897:-
699,17:41245766:CATATTGCTTATACTGCTGCTTATAGGTTCAGCT...,-9.789052,17_41245603_41246877:-
147,17:41203074:TCTTACCT:['ATGTTG'],-8.181610,17_41203080_41203134:-
946,17:41251791:C:['CCCAATTCAATGTAGACAGACGTCTTTTGA...,-7.988797,17_41251792_41251894:-


## Predict pathogenicity

In [9]:
dl = SplicingVCFDataloader(gtfIntervalTree, 
                          fasta,
                          vcf,
                          out_file=gtfIntervalTree,
                          split_seq=False, overhang=(50,10))

In [10]:
predictions = predict_all_table(model, dl, batch_size=1024, split_seq=False, assembly=True, pathogenicity=True)

2it [00:10,  5.25s/it]


In [11]:
predictionsMax = max_varEff(predictions)

In [12]:
predictionsMax.sort_values('mmsplice_diff', ascending=False).head()

,ID,mmsplice_diff,exons
577,17:41245766:CATATTGCTTATACTGCTGCTTATAGGTTCAGCT...,1.0,17_41245601_41246877:-
230,17:41215387:ACCCCTAAAGAGATCATAGA:['TATT'],1.0,17_41215377_41215390:-
712,17:41251814:CTTTTGAGGTTGTATCCGCTGCTTTGTCCTCAGA...,1.0,17_41251792_41251894:-
771,17:41256281:G:['C'],1.0,17_41256206_41256278:-
203,17:41215348:AC:['A'],1.0,17_41215350_41215390:-
